# Classification

In [6]:
import os
import sys
import matplotlib.pyplot as plt


from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import  DataLoader

from tqdm import tqdm
from sklearn.metrics import accuracy_score

### Configuration

In [7]:
sys.path.append('..')  # Replace '..' with the actual path to the parent directory containing 'src'.

# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

print(device)

cuda


In [8]:
from objects.dataset import CustomDataset
from objects.utils import split_dataset_1
from objects.models import Classifier

In [9]:
# Example usage:
root_directory = '../2d_data'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

custom_dataset = CustomDataset(root_directory, transform=transform)

train_set, test_set = split_dataset_1(custom_dataset, test_size=0.2)
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

In [10]:
# Initialize the classifier
num_classes = len(custom_dataset.classes)
classifier = Classifier(num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# Set the number of training epochs
num_epochs = 10

/home/yannt/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/yannt/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
# Lists to store training and test loss, and training and test accuracy values
train_loss_values = []
train_accuracy_values = []
test_loss_values = []
test_accuracy_values = []

# Training and testing loop
for epoch in range(num_epochs):
    # Training phase
    classifier.train()  # Set the model to training mode
    running_loss = 0.0
    predictions = []
    true_labels = []

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
      
        optimizer.zero_grad()  # Zero the gradients
        outputs = classifier(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

    # Calculate training accuracy
    train_accuracy = accuracy_score(true_labels, predictions)

    # Store training loss and accuracy values
    train_loss_values.append(running_loss / len(train_loader))
    train_accuracy_values.append(train_accuracy)

    # Print training loss and accuracy for each epoch
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss_values[-1]}, Training Accuracy: {train_accuracy * 100:.2f}%")

    # Testing phase
    classifier.eval()  # Set the model to evaluation mode
    test_running_loss = 0.0
    test_predictions = []
    test_true_labels = []

    
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = classifier(images)
            test_loss = criterion(outputs, labels)
            test_running_loss += test_loss.item()

            _, predicted = torch.max(outputs.data, 1)
            test_predictions.extend(predicted.cpu().numpy())
            test_true_labels.extend(labels.cpu().numpy())

    # Calculate test accuracy
    test_accuracy = accuracy_score(test_true_labels, test_predictions)

    # Store test loss and accuracy values
    test_loss_values.append(test_running_loss / len(test_loader))
    test_accuracy_values.append(test_accuracy)

    # Print test loss and accuracy for each epoch
    print(f"Epoch {epoch + 1}/{num_epochs}, Test Loss: {test_loss_values[-1]}, Test Accuracy: {test_accuracy * 100:.2f}%")

100%|██████████| 230/230 [00:43<00:00,  5.26it/s]


Epoch 1/10, Training Loss: 0.5752158862417159, Training Accuracy: 80.35%


100%|██████████| 58/58 [00:06<00:00,  9.46it/s]


Epoch 1/10, Test Loss: 2.001778639596084, Test Accuracy: 53.16%


100%|██████████| 230/230 [00:41<00:00,  5.57it/s]


...

Epoch 4/10, Training Loss: 0.14359901466447375, Training Accuracy: 94.93%


100%|██████████| 58/58 [00:06<00:00,  9.58it/s]


Epoch 4/10, Test Loss: 0.4286525609164402, Test Accuracy: 85.95%


 53%|█████▎    | 121/230 [00:22<00:20,  5.39it/s]


KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the test set
classifier.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = classifier(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")


In [ ]:
# Plot the learning curves
plt.plot(train_loss_values, label='Training Loss')
plt.plot(test_loss_values, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Test Loss')
plt.show()

In [ ]:
plt.plot(train_accuracy_values, label='Training Accuracy')
plt.plot(test_accuracy_values, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Test Accuracy')
plt.show()